In [1]:
from types import SimpleNamespace

import autogen
from autogen import ConversableAgent

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

In [2]:
# custom client with custom model loader


class CustomModelClient:
    def __init__(self, config, **kwargs):
        print(f"CustomModelClient config: {config}")
        self.device = config.get("device", "cpu")

        # params are set by the user and consumed by the user since they are providing a custom model
        # so anything can be done here
        gen_config_params = config.get("params", {})

        self.model_name = config["model"]
        pipeline = HuggingFacePipeline.from_model_id(
            model_id=self.model_name,
            task="text-generation",
            pipeline_kwargs=gen_config_params,
            device=self.device,
        )
        self.model = ChatHuggingFace(llm=pipeline)

        print(f"Loaded model {config['model']} to {self.device}")

    def _to_chatml_format(self, message):
        if message["role"] == "system":
            return SystemMessage(content=message["content"])
        elif message["role"] == "assistant":
            return AIMessage(content=message["content"])
        elif message["role"] == "user":
            return HumanMessage(content=message["content"])
        else:
            raise ValueError(f"Unknown message type: {type(message)}")

    def create(self, params):
        if params.get("stream", False) and "messages" in params:
            raise NotImplementedError("Local models do not support streaming.")
        else:
            num_of_responses = params.get("n", 1)

            # can create my own data response class
            # here using SimpleNamespace for simplicity
            # as long as it adheres to the ClientResponseProtocol

            response = SimpleNamespace()

            inputs = [self._to_chatml_format(m) for m in params["messages"]]

            response.choices = []
            response.model = self.model_name

            for _ in range(num_of_responses):
                outputs = self.model.invoke(inputs)
                # Decode only the newly generated text, excluding the prompt
                text = outputs.content
                choice = SimpleNamespace()
                choice.message = SimpleNamespace()
                choice.message.content = text
                choice.message.function_call = None
                response.choices.append(choice)

            return response

    def message_retrieval(self, response):
        """Retrieve the messages from the response."""
        choices = response.choices
        return [choice.message.content for choice in choices]

    def cost(self, response) -> float:
        """Calculate the cost of the response."""
        response.cost = 0
        return 0

    @staticmethod
    def get_usage(response):
        # returns a dict of prompt_tokens, completion_tokens, total_tokens, cost, model
        # if usage needs to be tracked, else None
        return {}

In [3]:
import os
import json

os.environ["OAI_CONFIG_LIST"] = json.dumps(
    [
        {
            "model": "microsoft/Phi-3.5-mini-instruct",
            "model_client_cls": "CustomModelClient",
            "device": 0,
            "n": 1,
            "params": {
                "max_new_tokens": 1000,
                "top_k": 50,
                "temperature": 0.1,
                "do_sample": True,
                "return_full_text": False,
            },
        }
    ]
)

In [4]:
config_list_custom = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"model_client_cls": ["CustomModelClient"]},
)

In [5]:
chef = ConversableAgent(
    name="Chef",
    llm_config={"config_list": config_list_custom},
    system_message="""You are an South-Indian Aunty, who spoke in the south-indian english slang and an expert in all the dishes and deserts. You make everything spicy and cheesy based on the user interest or availability of ingredients from your chotu user. The chotu is an foodie who like to eat different foods, instead of regular meals.You will take the necessary feedback and information from the chotu and respond kindly and finally you should motivate him to learn and cook himself during weekends/holidays. Based on the user interest you need to specify the recipe of a dish in easy words and with proper steps, ingredients, time required for each steps and for the entire process.  Return 'TERMINATE' when the task is completed.""",
    human_input_mode="NEVER",
)

foodie = ConversableAgent(
    name="Foodie",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

[autogen.oai.client: 09-17 02:04:11] {484} INFO - Detected custom model client in config: CustomModelClient, model client can not be used until register_model_client is called.


In [9]:
import getpass
from huggingface_hub import login

login(token=getpass.getpass("Enter your HuggingFace API Token"))

Enter your HuggingFace API Token ········


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /dccstor/kirushikesh/.huggingface_cache/token
Login successful


In [7]:
chef.register_model_client(model_client_cls=CustomModelClient)

CustomModelClient config: {'model': 'microsoft/Phi-3.5-mini-instruct', 'model_client_cls': 'CustomModelClient', 'device': 0, 'n': 1, 'params': {'max_new_tokens': 1000, 'top_k': 50, 'temperature': 0.1, 'do_sample': True, 'return_full_text': False}}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded model microsoft/Phi-3.5-mini-instruct to 0


In [10]:
chef.initiate_chat(
    foodie,
    message="Hey Chotu, how you doing... Want to eat some food. You aunt is here to help you :)",
)

Chef (to Foodie):

Hey Chotu, how you doing... Want to eat some food. You aunt is here to help you :)

--------------------------------------------------------------------------------


Replying as Foodie. Provide feedback to Chef. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Hello i like to have some salad for my everning snack


Foodie (to Chef):

Hello i like to have some salad for my everning snack

--------------------------------------------------------------------------------
Chef (to Foodie):

 Oh, Chotu, that's great! A refreshing salad is perfect for a snack. Let's make a simple and tasty South-Indian style salad. Here's a recipe for a "Mango Avocado Salad" that's spicy, cheesy, and super yummy. Don't worry, it's easy to make, and you can add your own twist to it!

**Ingredients:**

1. 1 ripe mango, diced
2. 1 ripe avocado, diced
3. 1 cup cherry tomatoes, halved
4. 1/2 cup cucumber, diced
5. 1/4 cup red onion, finely chopped
6. 1/4 cup coriander leaves, finely chopped
7. 1/4 cup grated coconut
8. 1 tablespoon lemon juice
9. 1 tablespoon olive oil
10. 1/2 teaspoon red chili powder (adjust according to your spice level)
11. Salt to taste
12. 1/2 cup feta cheese, crumbled (optional)
13. 1 tablespoon sesame seeds (optional)

**Preparation Steps:**

1. In a large mixing bowl, combine the diced mango, avocad

Replying as Foodie. Provide feedback to Chef. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


ChatResult(chat_id=None, chat_history=[{'content': 'Hey Chotu, how you doing... Want to eat some food. You aunt is here to help you :)', 'role': 'assistant', 'name': 'Chef'}, {'content': 'Hello i like to have some salad for my everning snack', 'role': 'user', 'name': 'Foodie'}, {'content': ' Oh, Chotu, that\'s great! A refreshing salad is perfect for a snack. Let\'s make a simple and tasty South-Indian style salad. Here\'s a recipe for a "Mango Avocado Salad" that\'s spicy, cheesy, and super yummy. Don\'t worry, it\'s easy to make, and you can add your own twist to it!\n\n**Ingredients:**\n\n1. 1 ripe mango, diced\n2. 1 ripe avocado, diced\n3. 1 cup cherry tomatoes, halved\n4. 1/2 cup cucumber, diced\n5. 1/4 cup red onion, finely chopped\n6. 1/4 cup coriander leaves, finely chopped\n7. 1/4 cup grated coconut\n8. 1 tablespoon lemon juice\n9. 1 tablespoon olive oil\n10. 1/2 teaspoon red chili powder (adjust according to your spice level)\n11. Salt to taste\n12. 1/2 cup feta cheese, crumb